## 프로젝트 기반 빅데이터 서비스 솔루션 개발 전문 과정

#### 교과목명 : 머신러닝응용
- 평가일 : 03.28
- 성명 : 권혁종
- 점수 :

문제 : LMEMBERS의 상품구매데이터를 이용하여 개인맞춤 상품 추천솔루션을 구축 후 다양한 활용 방안을 시현하세요. 

6개월단위로 데이터 분석을 진행하고 6~3개월전까지 많이 구입한 물품 중 3개월 이내로 구입하지 않은 물품을 추천과 동시에  
3개월안에 많이 구매한 물품을 knn이웃 방식과 잠재요인협엽 필터링 방식을 사용하여 추천아이템을 선정하여 추천

knn최근접이웃 과 잠재요인협업필터링의 두가지를 합쳐서 하이브리드 추천알고리즘 생성  
고객이 3개월 이전에 많이 구매했는데 최근 3개월동안 구매하지 않은 물품 추천  
1차프로젝트와 결합하여 관리대상고객 예측한 결과를 바탕으로 관리대상고객에게 상품을 추천하여 관리대상고객의 매출증대  
중분류 안에서 랜덤으로 소분류 하나를 선택하여 추천

In [1]:
import pandas as pd
data = pd.read_csv('../../lmembers/data/purprod2.csv',encoding='ms949')

In [2]:
data3 = data.loc[data.구매일자>=20151001]
data6 = data.loc[(data.구매일자<20151001)&(data.구매일자>=20150601)]

In [3]:
cate = pd.read_csv('../../lmembers/data/prodcl2.csv')

In [4]:
data3_cate = pd.merge(data3,cate,on='소분류코드')
data6_cate = pd.merge(data6,cate,on='소분류코드')

In [5]:
# 중분류명별 구매횟수를 기준으로 아이템 추천하기 위하여 피벗 생성
pdf3_ori = pd.pivot_table(data3_cate,
                     index = ['고객번호'],              # 행위치에 들어갈 열
                     columns = ['중분류명'],         # 열위치에 들어갈 열
                     values = ['영수증번호'],              # 데이터로 사용할 열
                     aggfunc = ['count'])         # 데이터 집계 함수
pdf6_ori = pd.pivot_table(data6_cate,
                     index = ['고객번호'],           
                     columns = ['중분류명'],       
                     values = ['영수증번호'],           
                     aggfunc = ['count'])    
pdf3=pdf3_ori.copy()
pdf6=pdf6_ori.copy()

In [6]:
pdf6_ori

count                                                          \
            영수증번호                                                           
중분류명  BABY/INFANT BOYS 1 BOYS 2 DIY욕실용품 EDUCATION 1 EDUCATION 2 Fast Food   
고객번호                                                                        
1             NaN    NaN    NaN     NaN         NaN         NaN       NaN   
2             NaN    NaN    NaN     NaN         NaN         NaN       NaN   
3             NaN    NaN    NaN     NaN         NaN         NaN       NaN   
4             NaN    NaN    NaN     NaN         NaN         NaN       NaN   
6             NaN    NaN    NaN     NaN         NaN         NaN       NaN   
...           ...    ...    ...     ...         ...         ...       ...   
19379         NaN    NaN    NaN     NaN         NaN         NaN       NaN   
19380         NaN    NaN    NaN     NaN         NaN         NaN       NaN   
19381         NaN    NaN    NaN     NaN         NaN         NaN       NaN   
19382         NaN    NaN    NaN     NaN         NaN         NaN       NaN   
19383         NaN    NaN    NaN     NaN         NaN         NaN       NaN   

                               ...                                             \
                               ...                                              
중분류명  GIRLS H&B선물세트 LOCAL김치채소  ... 홈웨어  홍인삼  화장지 화장지/티슈   화장품  황태 훼이셜케어 휴대폰용품   
고객번호                           ...                                              
1       NaN     NaN       NaN  ... NaN  NaN  NaN    NaN   2.0 NaN   NaN   NaN   
2       NaN     NaN       NaN  ... NaN  NaN  NaN    NaN  12.0 NaN   NaN   NaN   
3       NaN     NaN       NaN  ... NaN  NaN  3.0    NaN   NaN NaN   NaN   NaN   
4       NaN     NaN       NaN  ... NaN  NaN  NaN    NaN   1.0 NaN   NaN   NaN   
6       NaN     NaN       NaN  ... NaN  NaN  NaN    NaN   4.0 NaN   NaN   NaN   
...     ...     ...       ...  ...  ..  ...  ...    ...   ...  ..   ...   ...   
19379   NaN     NaN       NaN  ... NaN  NaN  NaN    1.0   NaN NaN   NaN   NaN   
19380   NaN     NaN       NaN  ... NaN  NaN  NaN    4.0   NaN NaN   NaN   NaN   
19381   NaN     NaN       NaN  ... NaN  4.0  NaN    NaN   NaN NaN   2.0   NaN   
19382   NaN     NaN       NaN  ... NaN  1.0  2.0    NaN   NaN NaN   NaN   NaN   
19383   NaN     NaN       NaN  ... NaN  NaN  NaN    2.0   NaN NaN   NaN   NaN   

                  
                  
중분류명   휴지통   흰우유  
고객번호              
1      NaN   NaN  
2      NaN   NaN  
3      NaN   NaN  
4      NaN   NaN  
6      2.0   1.0  
...    ...   ...  
19379  NaN   NaN  
19380  NaN   NaN  
19381  NaN  17.0  
19382  NaN   2.0  
19383  NaN   1.0  

[19319 rows x 540 columns]

# 최근접 이웃

In [7]:
# Nan값 0으로 대체 및 데이터프레임 정리
pdf3.fillna(0,inplace=True)
pdf3 = pdf3['count']['영수증번호']
pdf6.fillna(0,inplace=True)
pdf6 = pdf6['count']['영수증번호']

In [8]:
pdf3

중분류명,BABY/INFANT,BOYS 1,BOYS 2,DIY욕실용품,EDUCATION 1,EDUCATION 2,Fast Food,GIRLS,H&B선물세트,LOCAL김치채소,...,홈웨어,홍인삼,화장지,화장지/티슈,화장품,황태,훼이셜케어,휴대폰용품,휴지통,흰우유
고객번호,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
19380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0
19381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,6.0


In [9]:
# 코사인유사도는 행기준으로 작동하기에 transpose 함수로 전치
pdf_T = pdf3.transpose()
pdf_T

고객번호,1,2,3,4,5,6,7,8,9,10,...,19374,19375,19376,19377,19378,19379,19380,19381,19382,19383
중분류명,,,,,,,,,,,,,,,,,,,,,
BABY/INFANT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BOYS 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
BOYS 2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DIY욕실용품,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EDUCATION 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
황태,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
훼이셜케어,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
휴대폰용품,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# 코사인 유사도 산출 (행인 아이템들을 기준으로 사용자간의 유사도)
from sklearn.metrics.pairwise import cosine_similarity
pdf_sim = cosine_similarity(pdf_T, pdf_T)

In [11]:
# cosine_similarity()로 반환된 Numpy 행렬을 중분류명으로 매핑해 DataFrame으로 변환
pdf_sim_df = pd.DataFrame(data=pdf_sim, index=pdf3.columns,
                           columns=pdf3.columns)
print(pdf_sim_df.shape)
pdf_sim_df.head(3)

(539, 539)


중분류명,BABY/INFANT,BOYS 1,BOYS 2,DIY욕실용품,EDUCATION 1,EDUCATION 2,Fast Food,GIRLS,H&B선물세트,LOCAL김치채소,...,홈웨어,홍인삼,화장지,화장지/티슈,화장품,황태,훼이셜케어,휴대폰용품,휴지통,흰우유
중분류명,,,,,,,,,,,,,,,,,,,,,
BABY/INFANT,1.000000,0.197077,0.292339,0.034579,0.116243,0.392382,0.011274,0.341021,0.000000,0.016450,...,0.062216,0.025133,0.052006,0.000632,0.066941,0.044184,0.057444,0.048841,0.043725,0.142784
BOYS 1,0.197077,1.000000,0.205775,0.014724,0.272737,0.229297,0.021223,0.211933,0.002139,0.011429,...,0.073028,0.020847,0.052049,0.000992,0.058730,0.059660,0.044721,0.042382,0.022971,0.138893
BOYS 2,0.292339,0.205775,1.000000,0.044597,0.120748,0.291195,0.006107,0.249573,0.000000,0.013790,...,0.054956,0.019039,0.052123,0.000571,0.055936,0.042916,0.053801,0.037204,0.022950,0.137674


In [12]:
pdf_sim_df["황태"].sort_values(ascending=False)[:6]

중분류명
황태      1.000000
양념채소    0.328772
두부      0.323479
열매채소    0.323437
잎채소     0.318029
계란      0.313791
Name: 황태, dtype: float64

In [13]:
import numpy as np
# 실제 구매횟수와 코사인유사도를 이용하여 개인화된 예측구매횟수를 구함
def predict_rating(ratings_arr, item_sim_arr):
    ratings_pred = ratings_arr.dot(item_sim_arr)/ np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

In [14]:
ratings_pred_knn = predict_rating(pdf3.values , pdf_sim_df.values)
ratings_pred_matrix_knn = pd.DataFrame(data=ratings_pred_knn, index= pdf3.index,
                                   columns = pdf3.columns)
ratings_pred_matrix_knn.head(3)

중분류명,BABY/INFANT,BOYS 1,BOYS 2,DIY욕실용품,EDUCATION 1,EDUCATION 2,Fast Food,GIRLS,H&B선물세트,LOCAL김치채소,...,홈웨어,홍인삼,화장지,화장지/티슈,화장품,황태,훼이셜케어,휴대폰용품,휴지통,흰우유
고객번호,,,,,,,,,,,,,,,,,,,,,
1,0.178513,0.128338,0.145687,0.077211,0.147382,0.189609,0.052542,0.174465,0.072893,0.064148,...,0.065157,0.062938,0.175891,0.083574,0.732325,0.075566,0.077819,0.105106,0.131438,0.073298
2,0.295686,0.190292,0.226725,0.106015,0.228822,0.314455,0.046632,0.283588,0.114824,0.092574,...,0.081598,0.071194,0.146817,0.134221,1.374487,0.103666,0.103151,0.155894,0.229864,0.102722
3,0.122504,0.114167,0.124828,0.124925,0.119312,0.110169,0.102250,0.117901,0.071811,0.101240,...,0.106189,0.106762,0.543046,0.019858,0.215440,0.103956,0.114430,0.109148,0.087699,0.107324


In [15]:
from sklearn.metrics import mean_squared_error

# 고객이 구매한 물품에 대해서만 예측 성능 평가 MSE 를 구함. 
def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 인접 이웃 MSE: ', get_mse(ratings_pred_knn, pdf3.values ))


아이템 기반 모든 인접 이웃 MSE:  22.357388255100147


In [16]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)

    # 사용자-아이템 평점 행렬의 열 크기만큼 Loop 수행. 
    for col in range(ratings_arr.shape[1]):
        # 유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬의 index 반환
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        # 개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T) 
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))        
    return pred


In [17]:
ratings_pred_knn = predict_rating_topsim(pdf3.values , pdf_sim_df.values, n=20)
print('아이템 기반 인접 TOP-20 이웃 MSE: ', get_mse(ratings_pred_knn, pdf3.values ))


# 계산된 예측 구매 데이터는 DataFrame으로 재생성
ratings_pred_matrix_knn = pd.DataFrame(data=ratings_pred_knn, index= pdf3.index,
                                   columns = pdf3.columns)

C:\Users\kpo01\AppData\Local\Temp/ipykernel_22096/2382315358.py:11: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T)
C:\Users\kpo01\AppData\Local\Temp/ipykernel_22096/2382315358.py:12: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))


아이템 기반 인접 TOP-20 이웃 MSE:  15.593996655746523


In [18]:
user_rating_id = pdf3.loc[5064, :]
user_rating_id[ user_rating_id > 0].sort_values(ascending=False)[:10]

중분류명
쿠키케이크     26.0
맥주        10.0
견과류        9.0
라면         9.0
병통조림       8.0
우유         8.0
두채류        8.0
스낵         7.0
청소욕실용품     6.0
베이커리       6.0
Name: 5064, dtype: float64

In [19]:
def get_not_buy_list(ratings_matrix, custid):
    # custid로 입력받은 사용자의 모든 구매정보 추출하여 Series로 반환함. 
    # 반환된 user_rating 은 중분류명을 index로 가지는 Series 객체임. 
    user_rating = ratings_matrix.loc[custid,:]
    
    # user_rating이 0보다 크면 기존에 구매한 물품임. 대상 index를 추출하여 list 객체로 만듬
    already_buy = user_rating[ user_rating > 0].index.tolist()
    
    # 모든 중분류명을 list 객체로 만듬. 
    buy_list = ratings_matrix.columns.tolist()
    
    # 간편한 for문으로 already_buy에 해당하는 물품은 buy_list에서 제외함. 
    notbuy_list = [ item for item in buy_list if item not in already_buy]
    
    return notbuy_list

In [20]:
def recomm_item_by_custid(pred_df, custid, notbuy_list, top_n=10):
    # 구매예측 DataFrame에서 사용자id index와 notbuy_list로 들어온 중분류명 컬럼을 추출하여 가장 예측 구매횟수가 높은 순으로 정렬함. 
    recomm_items = pred_df.loc[custid, notbuy_list].sort_values(ascending=False)[:top_n]
    return recomm_items
    
# 사용자가 구매하지 않은 물품명 추출   
notbuy_list_knn = get_not_buy_list(pdf3, 5064)

# 아이템 기반의 인접 이웃 협업 필터링으로 물품 추천 
recomm_items_knn = recomm_item_by_custid(ratings_pred_matrix_knn, 5064, notbuy_list_knn, top_n=20)

# 구매 데이타를 DataFrame으로 생성. 
recomm_items_knn = pd.DataFrame(data=recomm_items_knn.values,index=recomm_items_knn.index,columns=['pred_score'])
recomm_items_knn

,pred_score
중분류명,
아이스크림,5.022117
과채음료,4.691159
기타음주류,4.544926
전통과자,4.473158
가공건식품,4.402297
즉석식품,4.400923
두발용품,4.376797
위생용품,4.295037
소주,4.287208


# 잠재요인 분석

In [21]:
import numpy as np
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적 곱으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P,Q.T)
    
    # 실제 R 행렬에서 NULL이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind,y_non_zero_ind]
    
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [22]:
# 행렬 분해
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda=0.01):
    num_users, num_items = R.shape
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size = (num_users,K))
    Q = np.random.normal(scale=1./K, size = (num_items,K))
    
    break_count = 0
    
    # R > 0인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장
    non_zeros = [(i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0]
    
    # SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
    for step in range(steps):  # steps는 SGD의 반복횟수
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i,:],Q[j,:].T)
            
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j,:] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i,:] - r_lambda*Q[j,:])
            
        rmse = get_rmse(R,P,Q, non_zeros)
        if ( step % 10) == 0: # 10회 반복할 때마다 오류 값 출력
            print(f'iteration step: {step}, rmse: {rmse}')
    return P, Q

In [23]:
# 구매횟수간 편차가 너무 커서 오류가 나서 log변환시킨값으로 예측 수행
pdf_log = pdf3.apply(lambda x: np.log1p(x))
pdf_log

중분류명,BABY/INFANT,BOYS 1,BOYS 2,DIY욕실용품,EDUCATION 1,EDUCATION 2,Fast Food,GIRLS,H&B선물세트,LOCAL김치채소,...,홈웨어,홍인삼,화장지,화장지/티슈,화장품,황태,훼이셜케어,휴대폰용품,휴지통,흰우유
고객번호,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,1.098612,0.0,0.000000,0.0,0.0,0.000000
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,2.197225,0.0,0.000000,0.0,0.0,0.000000
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000
5,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.693147,0.0,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19379,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,1.791759,0.000000,0.0,0.000000,0.0,0.0,0.000000
19380,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.693147,1.609438,0.693147,0.0,0.000000,0.0,0.0,0.000000
19381,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.693147,0.000000,0.000000,0.693147,0.0,0.693147,0.0,0.0,1.945910


In [25]:
P, Q = matrix_factorization(pdf_log.values, steps=200, K=50,learning_rate=0.001, r_lambda = 0.001)
pred_matrix = np.dot(P, Q.T)

iteration step: 0, rmse: 1.324315943427716
iteration step: 10, rmse: 0.5787775605469171
iteration step: 20, rmse: 0.4848604730618589
iteration step: 30, rmse: 0.4661639055082616
iteration step: 40, rmse: 0.45456017601928556
iteration step: 50, rmse: 0.4407071105905678
iteration step: 60, rmse: 0.4243306666250968
iteration step: 70, rmse: 0.40568201103179247
iteration step: 80, rmse: 0.3855742521903993
iteration step: 90, rmse: 0.3654916676001244
iteration step: 100, rmse: 0.3467774840983854
iteration step: 110, rmse: 0.32987498578003366
iteration step: 120, rmse: 0.314773177131604
iteration step: 130, rmse: 0.30136757800012315
iteration step: 140, rmse: 0.28953571629816477
iteration step: 150, rmse: 0.2791329517271167
iteration step: 160, rmse: 0.270008701533852
iteration step: 170, rmse: 0.2620248037904211
iteration step: 180, rmse: 0.2550548100410877
iteration step: 190, rmse: 0.2489776281659642


In [26]:
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index= pdf_log.index,
                                   columns = pdf_log.columns)

ratings_pred_matrix

중분류명,BABY/INFANT,BOYS 1,BOYS 2,DIY욕실용품,EDUCATION 1,EDUCATION 2,Fast Food,GIRLS,H&B선물세트,LOCAL김치채소,...,홈웨어,홍인삼,화장지,화장지/티슈,화장품,황태,훼이셜케어,휴대폰용품,휴지통,흰우유
고객번호,,,,,,,,,,,,,,,,,,,,,
1,0.824892,0.898407,0.740738,0.418425,0.749098,0.752011,0.362728,0.795916,0.266956,0.563633,...,0.605954,0.680232,0.623315,0.818059,1.066958,0.597535,0.604723,0.586282,0.555964,0.791693
2,0.999353,1.008811,0.993993,0.571195,0.911585,0.991316,0.512939,1.143379,0.389625,0.750261,...,0.830552,0.984922,1.112727,1.080499,2.132212,0.819430,0.876719,0.788315,0.774041,1.091003
3,1.089811,1.257830,0.995315,0.685513,1.040175,1.058542,0.613699,0.957585,0.460800,0.921148,...,0.901272,1.054115,0.850733,1.277249,1.051160,0.909681,0.955352,0.857599,0.817801,2.147967
4,0.928970,1.060098,0.891457,0.476492,0.841633,0.925491,0.414809,1.021240,0.317690,0.659479,...,0.718501,0.786422,0.763170,0.964112,1.220017,0.673798,0.736179,0.692835,0.644919,0.917904
5,0.758404,1.014602,0.825447,0.423866,0.778137,0.859223,0.385351,0.866118,0.313530,0.616167,...,0.621287,0.670430,0.262845,0.896166,0.727534,0.598784,0.639677,0.600885,0.578735,0.336734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19379,1.226209,1.458443,1.202157,0.846848,1.201525,1.250024,0.823277,1.159004,0.602319,1.147720,...,1.178765,1.286431,1.786461,1.639616,1.099784,1.116468,1.180467,1.125461,1.044113,2.903213
19380,1.340970,1.621142,1.397890,1.026837,1.349643,1.448034,0.955783,1.283539,0.726819,1.375802,...,1.304773,1.482216,0.742844,1.687678,0.840406,1.381869,1.361052,1.271283,1.212845,4.766357
19381,0.854834,1.004865,0.854604,0.594295,0.868918,0.835143,0.569582,0.976376,0.413194,0.852056,...,0.781879,0.934845,1.021416,1.061006,0.731818,0.860145,0.917073,0.801661,0.773654,1.942646


In [27]:
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id index와 unseen_list로 들어온 영화명 컬럼을 추출하여 가장 예측 평점이 높은 순으로 정렬 
    
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [28]:
# get_not_buy_list 재활용하여 사지않은 물품 반환  
notbuy_list = get_not_buy_list(pdf3, 5064)
notbuy_list

['BABY/INFANT',
 'BOYS 1',
 'BOYS 2',
 'DIY욕실용품',
 'EDUCATION 1',
 'EDUCATION 2',
 'Fast Food',
 'GIRLS',
 'H&B선물세트',
 'LOCAL김치채소',
 'LOCAL잎채소',
 'SPORTS/OUTDOOR',
 'TRAVEL',
 'VIDEOGAME',
 '가공건식품',
 '가공란',
 '가공양곡',
 '가공우유',
 '가구',
 '가루',
 '가방',
 '가방브랜드',
 '간식',
 '간식용과자',
 '간편요리',
 '간편조리',
 '간편조리행사',
 '감',
 '감귤',
 '개인위생용품',
 '건강가전',
 '건강기능식품',
 '건강선물세트',
 '건강식품',
 '건과일',
 '건어류',
 '건오징어',
 '건전지',
 '계란',
 '계절팩',
 '고등어/삼치',
 '고양이용품',
 '골프',
 '골프용품',
 '공구',
 '과자',
 '과채음료',
 '교육보조재',
 '교환/보수용품',
 '구강케어',
 '구기용품',
 '구두',
 '구색과일',
 '구이',
 '국내산계육',
 '국산게/갈치',
 '국산과일선물세트',
 '굴비',
 '굴비/반건생선선물세트',
 '규격RTC',
 '균일가',
 '그릇/수저',
 '기기',
 '기능성건강',
 '기능헤어케어',
 '기초화장품',
 '기타',
 '기타구색생선',
 '기타수산',
 '기타음주류',
 '기타의류',
 '기타조리식품',
 '기타축산',
 '기타회',
 '김/초밥',
 '김가공품',
 '김선물세트',
 '김치채소',
 '껌',
 '꽁치/가자미',
 '꿀',
 '나물',
 '나물류',
 '남성',
 '남성 트랜디',
 '남성베이직',
 '남성스포츠',
 '남성양말',
 '남성어덜트캐주얼',
 '남성용품',
 '남성이지캐주얼',
 '남성정장',
 '남성캐주얼',
 '남성케어',
 '냉동HMR',
 '냉동MS',
 '냉동간식',
 '냉동과일',
 '냉동규격생선',
 '냉동규격해물',
 '냉동만두',
 '냉장간식MS',
 '냉

In [29]:
# 아이템 기반의 인접 이웃 협업 필터링으로 영화 추천 
recomm_items_SGD = recomm_movie_by_userid(ratings_pred_matrix, 5064, notbuy_list, top_n=20)
recomm_items_SGD

중분류명
아이스크림      2.011234
소주         1.945464
기타음주류      1.600188
전통주        1.597904
냉장식사HMR    1.573598
발효유        1.514705
주유소        1.488670
캐주얼        1.414067
수입과자       1.359576
수산품        1.336576
고양이용품      1.292243
애견용품       1.284739
기초화장품      1.261446
시티웨어       1.229678
김치채소       1.222333
계란         1.213675
어묵맛살       1.206892
브랜드돼지고기    1.189413
국내산계육      1.185699
종량제봉투      1.153824
Name: 5064, dtype: float64

In [30]:
# 평점 데이타를 DataFrame으로 생성. 
recomm_items_SGD = pd.DataFrame(data=recomm_items_SGD.values,index=recomm_items_SGD.index,columns=['pred_score'])
recomm_items_SGD

,pred_score
중분류명,
아이스크림,2.011234
소주,1.945464
기타음주류,1.600188
전통주,1.597904
냉장식사HMR,1.573598
발효유,1.514705
주유소,1.488670
캐주얼,1.414067
수입과자,1.359576


In [31]:
recomm_items_knn

,pred_score
중분류명,
아이스크림,5.022117
과채음료,4.691159
기타음주류,4.544926
전통과자,4.473158
가공건식품,4.402297
즉석식품,4.400923
두발용품,4.376797
위생용품,4.295037
소주,4.287208


knn최근접이웃 과 잠재요인협업필터링의 두가지를 합쳐서 하이브리드 추천알고리즘 생성

In [32]:
# knn 과 잠재요인 협엽 필터링 두가지에 공통된 요소를 추천
recom_items = pd.merge(recomm_items_SGD,recomm_items_knn,on='중분류명')
recom_items

,pred_score_x,pred_score_y
중분류명,,
아이스크림,2.011234,5.022117
소주,1.945464,4.287208
기타음주류,1.600188,4.544926
전통주,1.597904,3.921326
발효유,1.514705,4.195750


In [33]:
recom_items['recom_pred'] = recom_items.iloc[:,1]*recom_items.iloc[:,0]
recom_items = recom_items.sort_values(by='recom_pred',ascending=False)

In [34]:
recom_items

,pred_score_x,pred_score_y,recom_pred
중분류명,,,
아이스크림,2.011234,5.022117,10.100650
소주,1.945464,4.287208,8.340607
기타음주류,1.600188,4.544926,7.272738
발효유,1.514705,4.195750,6.355321
전통주,1.597904,3.921326,6.265903


고객이 3개월 이전에 많이 구매했는데 최근 3개월동안 구매하지 않은 물품 추천

In [35]:
# 6~3개월전까지 많이 구입한 물품 중 3개월 이내로 구입하지 않은 물품을 추천
def recom_notbuy(custid,pdf3,pdf6):
    user_rating = pdf6.loc[custid,:]
    already_buy = user_rating[ user_rating > 0].sort_values(ascending=False)
    notbuy_list = get_not_buy_list(pdf3, custid)
    for x in already_buy.index:
        if x in notbuy_list:
            already_buy.drop(index=x ,axis=0,inplace=True)
    recom_list = already_buy
    return recom_list


In [36]:
recom_notbuy(546,pdf3,pdf6)

중분류명
농산물      26.0
가공식품     25.0
명품       12.0
젓갈/반찬     8.0
화장품       6.0
주방용품      4.0
섬유잡화      3.0
구두        3.0
차/커피      2.0
수산품       2.0
건강식품      1.0
Name: 546, dtype: float64

1차프로젝트와 결합하여 관리대상고객 예측한 결과를 바탕으로 관리대상고객에게 상품을 추천하여 관리대상고객의 매출증대

In [37]:
def recommend_list(custid,pdf3,pdf6,top = 50,N=5):
    notbuy_list = get_not_buy_list(pdf3, custid)
    recomm_items_knn = recomm_item_by_custid(ratings_pred_matrix_knn, custid, notbuy_list, top_n=top)
    recomm_items_knn = pd.DataFrame(data=recomm_items_knn.values,index=recomm_items_knn.index,columns=['pred_score'])
    recomm_items_SGD = recomm_movie_by_userid(ratings_pred_matrix, custid, notbuy_list, top_n=top)
    recomm_items_SGD = pd.DataFrame(data=recomm_items_SGD.values,index=recomm_items_SGD.index,columns=['pred_score'])
    recom_items = pd.merge(recomm_items_SGD,recomm_items_knn,on='중분류명')
    recom_items['recom_pred'] = recom_items.iloc[:,1]*recom_items.iloc[:,0]
    recom_items = recom_items.sort_values(by='recom_pred',ascending=False)
    recom_list = recom_notbuy(custid,pdf3,pdf6)
    print(f'3개월전에 {recom_list.index.values[:N]}물품들을 구매하셨는데 지금은 필요하지 않으신가요?')
    print()
    print(f'{custid}고객님이 구매하신 상품을 구매한 고객들이 많이 구매한 {recom_items.index.values[:N]}은/는 어떠세요?')

In [67]:
# 감소고객의 리스트를 뽑아온뒤 여기에 넣어서 추천을 하면 된다
recommend_list(1234,pdf3,pdf6)

3개월전에 ['열매채소' '양념채소' '잎채소' '닭고기' '나물']물품들을 구매하셨는데 지금은 필요하지 않으신가요?

1234고객님이 구매하신 상품을 구매한 고객들이 많이 구매한 ['종량제봉투' '흰우유' '샐러드채소' '어묵맛살' '건과']은/는 어떠세요?


중분류 안에서 랜덤으로 소분류 하나를 선택하여 추천

In [64]:
import numpy as np
def recommend_more_list(custid,pdf3,pdf6,top = 50,N=5):
    global cate
    notbuy_list = get_not_buy_list(pdf3, custid)
    recomm_items_knn = recomm_item_by_custid(ratings_pred_matrix_knn, custid, notbuy_list, top_n=top)
    recomm_items_knn = pd.DataFrame(data=recomm_items_knn.values,index=recomm_items_knn.index,columns=['pred_score'])
    recomm_items_SGD = recomm_movie_by_userid(ratings_pred_matrix, custid, notbuy_list, top_n=top)
    recomm_items_SGD = pd.DataFrame(data=recomm_items_SGD.values,index=recomm_items_SGD.index,columns=['pred_score'])
    recom_items = pd.merge(recomm_items_SGD,recomm_items_knn,on='중분류명')
    recom_items['recom_pred'] = recom_items.iloc[:,1]*recom_items.iloc[:,0]
    recom_items = recom_items.sort_values(by='recom_pred',ascending=False)
    recom_list = recom_notbuy(custid,pdf3,pdf6)
    more_list = []
    more_list2 = []
    for i in recom_list.index.values[:N]:
        more_list.append(np.random.choice(cate.소분류명[cate.중분류명==i]))
    for i in recom_items.index.values[:N]:
        more_list2.append(np.random.choice(cate.소분류명[cate.중분류명==i]))
    print(f'3개월전에 {recom_list.index.values[:N]}물품들을 구매하셨는데 혹시 {more_list}는 어떠신가요?')
    print()
    print(f'{custid}고객님이 구매하신 상품을 구매한 고객들이 많이 구매한 {more_list2}은/는 어떠세요?')

In [68]:
recommend_more_list(1234,pdf3,pdf6)

3개월전에 ['열매채소' '양념채소' '잎채소' '닭고기' '나물']물품들을 구매하셨는데 혹시 ['피망', '건고추', '죽순', '오골계', '고사리']는 어떠신가요?

1234고객님이 구매하신 상품을 구매한 고객들이 많이 구매한 ['재사용봉투', '기능성시유', '양상추', '곤약', '곶감']은/는 어떠세요?
